## Download Radolan .bin.gz files from DWD ftp server

To-Do
Ordner im System erstellen mit der jeweiligen Jahreszahl der Daten klappts?
Nicht die Dateien Downloaden die bereits lokal im Ordner sind 
        '''Create directory structure if it doesn't exist'''
    if not os.path.exists(os.path.dirname(key)):
        os.makedirs(os.path.dirname(key))
Entpacken?

In [1]:
#!pip install pyftpdlib
import ftplib
import os, os.path
from pathlib import Path
import tarfile #Check if tarfile is corrupted, important when script is run twice 
BLOCK_SIZE = 1024

#working dir
windows = "C:/Users/Elisabeth/Desktop"
ubuntu = "/home/elisabeth/Dokumente/master/radolan_data/raw"
download_directory = "radolan_downloads" #Directory where files will be saved
os.chdir(windows) 
if not os.path.exists(download_directory):
    os.makedirs(download_directory)
os.chdir(download_directory)
print("Actual Working dir: ", os.getcwd())

#location of the file in the ftp server
path = 'climate_environment/CDC/grids_germany/daily/radolan/historical/bin/'
host = "https://opendata.dwd.de/"
ip =  "141.38.2.26" 

#Create Connection
ftp = ftplib.FTP(ip) 

#Login
ftp.login()

#Change Path to Radolan bin directory
ftp.cwd(path)

#Get list of Years available 
paths = ftp.nlst()
print("Available Years: ", paths)

#Create local directories
for path in paths:
    if not os.path.exists(path):
        os.makedirs(path)

#Workaround for problem that if i cahnged first time i am not in right directory
count = True
#For loop to download all files from every year directory 
for path in paths:
    if count == True:
        count = False 
        ftp.cwd(path)
        print("Path: ", ftp.pwd())
        filenames = ftp.nlst()
        print("Files stored in this path: ", filenames)
        os.chdir(path)
        for file in filenames:
            print("Start downloading file: ", file)
            if os.path.isfile(file) and os.access(file, os.R_OK)== True:
                print(file, "already exists!")
                continue
            else:
                handle = open(file.lstrip("/"), 'wb') 
                ftp.retrbinary('RETR %s' % file, handle.write)
                print("File " + file + " downloaded succsefully!")
                continue
        os.chdir('../')
    else:
        ftp.cwd(("../"))
        ftp.cwd(path)
        count = False 
        print("Path: ", ftp.pwd())
        print("Working dir : ", os.getcwd())
        filenames = ftp.nlst()
        print("Files stored in this path: ", filenames)
        os.chdir(path)
        for file in filenames:
            print("Start downloading file: ", file)
            if os.path.isfile(file) and os.access(file, os.R_OK) == True:
                print(file, "already exists!")
                continue
            else:
                handle = open(file.lstrip("/"), 'wb') 
                ftp.retrbinary('RETR %s' % file, handle.write)
                print("File " + file + " downloaded succsefully!")
                continue
        os.chdir('../')

                    
print("Job done... connection closed")
#close connection
ftp.quit()


Actual Working dir:  C:\Users\Elisabeth\Desktop\radolan_downloads
Available Years:  ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']
Path:  /climate_environment/CDC/grids_germany/daily/radolan/historical/bin/2006
Files stored in this path:  ['SF-200610.tar.gz', 'SF-200611.tar.gz', 'SF-200612.tar.gz']
Start downloading file:  SF-200610.tar.gz
SF-200610.tar.gz already exists!
Start downloading file:  SF-200611.tar.gz
SF-200611.tar.gz already exists!
Start downloading file:  SF-200612.tar.gz
SF-200612.tar.gz already exists!
Path:  /climate_environment/CDC/grids_germany/daily/radolan/historical/bin/2007
Working dir :  C:\Users\Elisabeth\Desktop\radolan_downloads
Files stored in this path:  ['SF-200701.tar.gz', 'SF-200702.tar.gz', 'SF-200703.tar.gz', 'SF-200704.tar.gz', 'SF-200705.tar.gz', 'SF-200706.tar.gz', 'SF-200707.tar.gz', 'SF-200708.tar.gz', 'SF-200709.tar.gz', 'SF-200710.tar.gz', 'SF-200711.tar.gz', 'SF-200712.tar.gz']
Start downloadin

File SF201206.tar.gz downloaded succsefully!
Start downloading file:  SF201207.tar.gz
File SF201207.tar.gz downloaded succsefully!
Start downloading file:  SF201208.tar.gz
File SF201208.tar.gz downloaded succsefully!
Start downloading file:  SF201209.tar.gz
File SF201209.tar.gz downloaded succsefully!
Start downloading file:  SF201210.tar.gz
File SF201210.tar.gz downloaded succsefully!
Start downloading file:  SF201211.tar.gz
File SF201211.tar.gz downloaded succsefully!
Start downloading file:  SF201212.tar.gz
File SF201212.tar.gz downloaded succsefully!
Path:  /climate_environment/CDC/grids_germany/daily/radolan/historical/bin/2013
Working dir :  C:\Users\Elisabeth\Desktop\radolan_downloads
Files stored in this path:  ['SF201301.tar.gz', 'SF201302.tar.gz', 'SF201303.tar.gz', 'SF201304.tar.gz', 'SF201305.tar.gz', 'SF201306.tar.gz', 'SF201307.tar.gz', 'SF201308.tar.gz', 'SF201309.tar.gz', 'SF201310.tar.gz', 'SF201311.tar.gz', 'SF201312.tar.gz']
Start downloading file:  SF201301.tar.gz
F

'221 Goodbye.'

In [ ]:
#!pip install pyftpdlib
import ftplib
import os, os.path
from pathlib import Path
import tarfile #Check if tarfile is corrupted, important when script is run twice 
BLOCK_SIZE = 1024

#working dir
windows = "C:/Users/Elisabeth/Desktop"
ubuntu = "/home/elisabeth/Dokumente/master/radolan_data/raw"
download_directory = "radolan_downloads" #Directory where files will be saved
os.chdir(windows) 
if not os.path.exists(download_directory):
    os.makedirs(download_directory)
os.chdir(download_directory)
print("Actual Working dir: ", os.getcwd())

#location of the file in the ftp server
path = 'climate_environment/CDC/grids_germany/daily/radolan/historical/bin/'
host = "https://opendata.dwd.de/"
ip =  "141.38.2.26" 

#Create Connection
ftp = ftplib.FTP(ip) 

#Login
ftp.login()

#Change Path to Radolan bin directory
ftp.cwd(path)

#Get list of Years available 
paths = ftp.nlst()
print("Available Years: ", paths)

#Create local directories
for path in paths:
    if not os.path.exists(path):
        os.makedirs(path)
   

#Workaround for problem that if i cahnged first time i am not in right directory
count = True
#For loop to download all files from every year directory 
for path in paths:
    if count == True:
        count = False 
        ftp.cwd(path)
        print("Path: ", ftp.pwd())
        filenames = ftp.nlst()
        print("Files stored in this path: ", filenames)
        os.chdir(path)
        for file in filenames:
            print("Check if file is available on disk: ", file)
            try: 
                tardude = tarfile.open(file)
            except:
                print( "There was an error opening tarfile. The file might be corrupt or missing start new download...")
                handle = open(file.lstrip("/"), 'wb') 
                ftp.retrbinary('RETR %s' % file, handle.write)
                print("File " + file + " downloaded succsefully!")
            try:
                members = tardude.getmembers()
            except:
                print("Error reading tarfile membrs start new download...")
                handle = open(file.lstrip("/"), 'wb') 
                ftp.retrbinary('RETR %s' % file, handle.write)
                print("File " + file + " downloaded succsefully!")
            for member_info in members:
                try:
                    check = tardude.extractfile(member_info.name)
                    while 1:
                        data = check.read(BLOCK_SIZE)
                        if not data:
                            break
                except:
                    print("Tar file is corrupt, start downloading again...")
                    handle = open(file.lstrip("/"), 'wb') 
                    ftp.retrbinary('RETR %s' % file, handle.write)
                    print("File " + file + " downloaded succsefully!")
        os.chdir('../')
    else:
        ftp.cwd(("../"))
        ftp.cwd(path)
        count = False 
        print("Path: ", ftp.pwd())
        print("Working dir : ", os.getcwd())
        filenames = ftp.nlst()
        print("Files stored in this path: ", filenames)
        os.chdir(path)
        for file in filenames:
            print("Check if file is available on disk: ", file)
            try: 
                tardude = tarfile.open(file)
            except:
                print( "There was an error opening tarfile. The file might be corrupt or missing start new download...")
                handle = open(file.lstrip("/"), 'wb') 
                ftp.retrbinary('RETR %s' % file, handle.write)
                print("File " + file + " downloaded succsefully!")
            try:
                members = tardude.getmembers()
            except:
                print("Error reading tarfile membrs start new download...")
                handle = open(file.lstrip("/"), 'wb') 
                ftp.retrbinary('RETR %s' % file, handle.write)
                print("File " + file + " downloaded succsefully!")
            for member_info in members:
                try:
                    check = tardude.extractfile(member_info.name)
                    while 1:
                        data = check.read(BLOCK_SIZE)
                        if not data:
                            break
                except:
                    print("Tar file is corrupt, start downloading again...")
                    handle = open(file.lstrip("/"), 'wb') 
                    ftp.retrbinary('RETR %s' % file, handle.write)
                    print("File " + file + " downloaded succsefully!")
        os.chdir('../')

                    
print("Job done... connection closed")
#close connection
ftp.quit()


Actual Working dir:  C:\Users\Elisabeth\Desktop\radolan_downloads
Available Years:  ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']
Path:  /climate_environment/CDC/grids_germany/daily/radolan/historical/bin/2006
Files stored in this path:  ['SF-200610.tar.gz', 'SF-200611.tar.gz', 'SF-200612.tar.gz']
Check if file is available on disk:  SF-200610.tar.gz
Check if file is available on disk:  SF-200611.tar.gz
Check if file is available on disk:  SF-200612.tar.gz
Path:  /climate_environment/CDC/grids_germany/daily/radolan/historical/bin/2007
Working dir :  C:\Users\Elisabeth\Desktop\radolan_downloads
Files stored in this path:  ['SF-200701.tar.gz', 'SF-200702.tar.gz', 'SF-200703.tar.gz', 'SF-200704.tar.gz', 'SF-200705.tar.gz', 'SF-200706.tar.gz', 'SF-200707.tar.gz', 'SF-200708.tar.gz', 'SF-200709.tar.gz', 'SF-200710.tar.gz', 'SF-200711.tar.gz', 'SF-200712.tar.gz']
Check if file is available on disk:  SF-200701.tar.gz
Error reading tarfile mem